# Group 0 - Diane Woodbridge, Precipitation and Crimes in San Francsico

In [0]:
from datetime import datetime

import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
spark = SparkSession.builder.getOrCreate()

### Load data from MongoDB and Create features/label

In [0]:
database = 'msds697'
collection = 'sf_record'
user_name = 'admin'
password = 'msds697'
address = 'msds697.qgylt.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

In [0]:
df = spark.read.format("mongo").option("uri",connection_string).load()

In [0]:
df.show()

+---------------------------+--------+--------------------+---------------------+----------+--------------------+---------------+--------------------+---------------------+------------------+-----------------------+------------------------+--------------------+--------------------+--------------------+--------------------+-----------+--------------------+--------------------+---------------+--------------------+--------------------+--------------------+-----------+---------------+--------------+----------------+--------------------+--------------+-------------------+-------+
|:@computed_region_ajp5_b2md|     _id|              agency|analysis_neighborhood|cad_number|call_last_updated_at|call_type_final|call_type_final_desc|call_type_final_notes|call_type_original|call_type_original_desc|call_type_original_notes|      close_datetime|          data_as_of|      data_loaded_at|   dispatch_datetime|disposition|    enroute_datetime|      entry_datetime|intersection_id|   intersection_name|  i

In [0]:
filtered_df = df.select("received_datetime", "intersection_id", "agency", "call_type_final_desc", "sensitive_call", "weather").filter(size("weather") > 0).filter("intersection_id is not null")

In [0]:
def max_weather(x):
    x = np.array(x)
    x_list = x.tolist()
    
    max = 0
    for elem in x_list:
        if float(elem[-1]) > max:
            max = float(elem[-1])
    return max
   
udf_max_weather = udf(max_weather, FloatType())

In [0]:
def convert_str_to_hour(x):
    timestamp = datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.%f')
    return timestamp

udf_convert_str_to_hour = udf(convert_str_to_hour, TimestampType())

In [0]:
calls_weather = filtered_df.select(hour(udf_convert_str_to_hour("received_datetime")).alias("hour"),
                                   "intersection_id",
                                   "agency",
                                   "call_type_final_desc",
                                   "sensitive_call",
                                   udf_max_weather("weather").alias("max_precipitation"))

In [0]:
calls_weather.show(truncate=False)

+----+---------------+-------------------------------+--------------------+--------------+-----------------+
|hour|intersection_id|agency                         |call_type_final_desc|sensitive_call|max_precipitation|
+----+---------------+-------------------------------+--------------------+--------------+-----------------+
|21  |23798000       |Police                         |WELL BEING CHECK    |False         |366.0            |
|15  |23932000       |Police                         |FIGHT NO WEAPON     |False         |366.0            |
|13  |27216000       |Police                         |WELL BEING CHECK    |False         |366.0            |
|20  |20618000       |Police                         |RECOVER MISSING VEH |False         |366.0            |
|11  |33288000       |Police                         |MISC                |False         |366.0            |
|16  |24314000       |Police                         |SUSPICIOUS PERSON   |False         |366.0            |
|10  |24557000     

In [0]:
hourly_count = calls_weather.groupBy("hour","max_precipitation","agency").count().cache()

In [0]:
hourly_count.show()

+----+-----------------+--------------------+-----+
|hour|max_precipitation|              agency|count|
+----+-----------------+--------------------+-----+
|   1|            366.0|              Police|   18|
|  16|             33.0|              Police|   43|
|   7|            180.0|              Police|   35|
|   9|            180.0|             Sheriff|    2|
|  19|             89.0|              Police|   47|
|  15|            366.0|             Sheriff|    1|
|   6|            366.0|Municipal Transpo...|    3|
|   5|            554.0|Municipal Transpo...|    1|
|   9|             89.0|              Police|   31|
|   5|            439.0|              Police|   20|
|  18|             94.0|               Other|    1|
|  13|             10.0|             Sheriff|    5|
|   4|            180.0|              Police|    8|
|   8|             91.0|             Sheriff|    5|
|  12|             10.0|             Sheriff|    2|
|  15|             10.0|             Sheriff|    3|
|  17|      

###String Indexer for columns

In [0]:
def indexStringColumns(df, cols):
    # variable newdf will be updated several times
    newdf = df
    
    for c in cols:
        # For each given colum, fits StringIndexerModel.
        si = StringIndexer(inputCol=c, outputCol=c+"-num").setHandleInvalid("keep")
        sm = si.fit(newdf)
        
        # Creates a DataFame by putting the transformed values in the new colum with suffix "-num" 
        # and then drops the original columns.
        # and drop the "-num" suffix. 
        newdf = sm.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-num", c)
    return newdf

In [0]:
categorical_cols = ["hour","agency"]
sti_df = indexStringColumns(hourly_count, categorical_cols)

### One Hot Encode features

In [0]:
def oneHotEncodeColumns(df, cols):
    newdf = df
    for c in cols:
        # For each given colum, create OneHotEncoder. 
        # dropLast : Whether to drop the last category in the encoded vector (default: true)
        ohe = OneHotEncoder(inputCol=c, outputCol=c+"-onehot", dropLast=False)
        ohe_model = ohe.fit(newdf)
        #Creates a DataFame by putting the transformed values in the new colum with suffix "-onehot" 
        #and then drops the original columns.
        #and drop the "-onehot" suffix. 
        newdf = ohe_model.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-onehot", c)
    return newdf

In [0]:
def removeColFromArray(cols, col):
    new_cols = cols.copy()
    if (col in cols):
        new_cols.remove(col)
    return new_cols

In [0]:
ohi_df = oneHotEncodeColumns(sti_df, categorical_cols)

In [0]:
ohi_df.show()

+-----------------+-----+---------------+-------------+
|max_precipitation|count|           hour|       agency|
+-----------------+-----+---------------+-------------+
|            366.0|   18|(25,[15],[1.0])|(5,[0],[1.0])|
|             33.0|   43|(25,[12],[1.0])|(5,[0],[1.0])|
|            180.0|   35| (25,[8],[1.0])|(5,[0],[1.0])|
|            180.0|    2| (25,[4],[1.0])|(5,[2],[1.0])|
|             89.0|   47|(25,[17],[1.0])|(5,[0],[1.0])|
|            678.0|   27| (25,[0],[1.0])|(5,[1],[1.0])|
|            366.0|    1| (25,[7],[1.0])|(5,[2],[1.0])|
|            366.0|    3| (25,[9],[1.0])|(5,[1],[1.0])|
|            554.0|    1|(25,[14],[1.0])|(5,[1],[1.0])|
|             89.0|   31| (25,[4],[1.0])|(5,[0],[1.0])|
|            678.0|   17| (25,[2],[1.0])|(5,[1],[1.0])|
|            439.0|   20|(25,[14],[1.0])|(5,[0],[1.0])|
|             94.0|    1|(25,[16],[1.0])|(5,[3],[1.0])|
|             10.0|    5| (25,[0],[1.0])|(5,[2],[1.0])|
|            180.0|    8|(25,[23],[1.0])|(5,[0],

### Create a dataframe with features and label

In [0]:
va = VectorAssembler(outputCol="features", inputCols=["max_precipitation", "hour", "agency"])
va_df = va.transform(ohi_df).select("features", "count").withColumnRenamed("count", "label")

In [0]:
splits = va_df.randomSplit([0.8, 0.2])

In [0]:
train = splits[0].cache()
validation = splits[1].cache()

## ML Goals (Diane Woodbridge)
Develop a regression model to predict the number of crimes based on the time of the day, agency and precipitation.

## ML Outcome
Was able to develop a model to predict the number of crimes/incidences using Random Forest Regressor and yield RMSE of 5.12.

### Train model without/with cross validation

In [0]:
# witout CV
rf = RandomForestRegressor(maxDepth=10)
rfmodel = rf.fit(train)

In [0]:
# with CV

rf = RandomForestRegressor()
reval = RegressionEvaluator()
cv = CrossValidator().setEstimator(rf).setEvaluator(reval).setNumFolds(5)

#ParamGridBuilder() – combinations of parameters and their values.
paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [5, 10, 15, 20])\
                              .addGrid(rf.numTrees, [10, 20, 30]).build()

#setEstimatorParamMaps() takes ParamGridBuilder().
cv.setEstimatorParamMaps(paramGrid)
cvmodel = cv.fit(train)


### Validate the model using validation set.

In [0]:
# without CV
validpredicts = rfmodel.transform(validation)
print(validpredicts.show())
reval = RegressionEvaluator()
print (reval.getMetricName() +":" + str(reval.evaluate(validpredicts)))

+--------------------+-----+------------------+
|            features|label|        prediction|
+--------------------+-----+------------------+
|(31,[0,15,27],[55...|    1| 6.098390241956728|
|(31,[0,15,26],[43...|   20|16.242162465670532|
|(31,[0,17,29],[94...|    1| 4.621136858783918|
|(31,[0,17,27],[25...|   17|15.805116198685345|
|(31,[0,3,28],[10....|    2|2.7892117248613966|
|(31,[0,15,26],[36...|   15|16.242162465670532|
|(31,[0,22,26],[67...|   25| 18.53850510027879|
|(31,[0,8,28],[25....|    1|1.8731791391520862|
|(31,[0,15,27],[43...|    1| 6.098390241956728|
|(31,[0,20,26],[25...|   30|33.780559011110164|
|(31,[0,22,27],[55...|    2| 13.38651818791104|
|(31,[0,2,26],[366...|   47| 36.66521133146994|
|(31,[0,4,28],[300...|    1|2.0542422674868863|
|(31,[0,5,28],[366...|    3|1.9047467830909202|
|(31,[0,6,27],[300...|   22|23.317014747950637|
|(31,[0,19,28],[67...|    1|1.1727899531024533|
|(31,[0,12,27],[25...|    3| 3.299017943254684|
|(31,[0,18,26],[43...|   34|33.898869687

In [0]:
# with CV
RegressionEvaluator().evaluate(cvmodel.bestModel.transform(validation))

Out[25]: 5.121427747912555

### Store the feature and label dataframe to mongodb for future use

In [0]:
def sparseToDenseArray(sparse_array):
    return sparse_array.toArray().tolist()

udf_sparse_dense_array = udf(sparseToDenseArray, ArrayType(FloatType()))
va_df_dense_v_to_array =  va_df.select(udf_sparse_dense_array(va_df["features"]).alias("features"), va_df["label"])

In [0]:
va_df_dense_v_to_array.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[366.0, 0.0, 0.0,...|   18|
|[33.0, 0.0, 0.0, ...|   43|
|[180.0, 0.0, 0.0,...|   35|
|[180.0, 0.0, 0.0,...|    2|
|[89.0, 0.0, 0.0, ...|   47|
|[678.0, 1.0, 0.0,...|   27|
|[366.0, 0.0, 0.0,...|    1|
|[366.0, 0.0, 0.0,...|    3|
|[554.0, 0.0, 0.0,...|    1|
|[89.0, 0.0, 0.0, ...|   31|
|[678.0, 0.0, 0.0,...|   17|
|[439.0, 0.0, 0.0,...|   20|
|[94.0, 0.0, 0.0, ...|    1|
|[10.0, 1.0, 0.0, ...|    5|
|[180.0, 0.0, 0.0,...|    8|
|[91.0, 0.0, 0.0, ...|    5|
|[25.0, 0.0, 0.0, ...|   17|
|[10.0, 0.0, 0.0, ...|    2|
|[10.0, 0.0, 0.0, ...|    3|
|[33.0, 0.0, 0.0, ...|   53|
+--------------------+-----+
only showing top 20 rows



In [0]:
database = 'msds697'
collection = 'sf_record_feature'
user_name = 'admin'
password = 'msds697'
address = 'msds697.qgylt.mongodb.net'
connection_string = f"mongodb+srv://{user_name}:{password}@{address}/{database}.{collection}"

In [0]:
va_df_dense_v_to_array.write.format("com.mongodb.spark.sql.DefaultSource")\
                     .mode("overwrite")\
                     .option("uri", connection_string)\
                     .save()